In [ ]:
#Import dependencies
import cv2
from PIL import Image
import os
import random
import numpy as np
from matplotlib import pyplot as plt

#Import tensorflow dependencies
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def make_embedding():
    input = Input(shape=(224,224,3), name='flood_image')

    #First convulutional layer
    c1 = Conv2D(64, (10,10), activation='relu')(input)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)

    #Second convulutional layer
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)

    #Third and final convulutional layer
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)

    #Final layer
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)

    return Model(inputs=[input], outputs=[d1], name='embedding')

In [ ]:
embedding = make_embedding()
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flood_image (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 215, 215, 64)      19264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 108, 108, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 102, 102, 128)     401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 51, 51, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 128)       26

In [ ]:
# Siamese L1 Distance class
class L1Dist(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    # Similarity calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [ ]:
def make_siamese_model(): 
    
    # Anchor image input in the network
    input_image = Input(name='input_img', shape=(224,224,3))
    
    # Validation image in the network 
    validation_image = Input(name='validation_img', shape=(224,224,3))
    
    # Combine siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification layer 
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [ ]:
# Setup paths
TRUE_PAIRINGS = '/content/drive/MyDrive/Third_Year_Project/Siamese_Network/Data_Pre_Processing_Siamese/Image_Pairs_Equal'
FALSE_PAIRINGS = '/content/drive/MyDrive/Third_Year_Project/Siamese_Network/Data_Pre_Processing_Siamese/Image_Pairs'

import tensorflow.compat.v1 as tf1

#Get our anchor, negative and positive data
#This is done by creating an iterator for this path up to file 500
pairs_true = tf.data.Dataset.list_files(TRUE_PAIRINGS +'/*').take(150)
pairs_false = tf.data.Dataset.list_files(FALSE_PAIRINGS +'/*').take(150)

def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 224x224x3
    img = tf.image.resize(img, (224,224))
    # Scale image to be between 0 and 1 
    img = img / 255.0

    # Return image
    return img

positives = tf.data.Dataset.zip((pairs_true, tf.data.Dataset.from_tensor_slices(tf.ones(len(pairs_true)))))
negatives = tf.data.Dataset.zip((pairs_false, tf.data.Dataset.from_tensor_slices(tf.zeros(len(pairs_false)))))
data = positives.concatenate(negatives)

samples = data.as_numpy_iterator()

In [ ]:
def preprocess_twin(directory, label):
    
    count = 1
    for filename in os.listdir(directory):

        if count == 1:
          input_img = directory.decode() + '/' + filename.decode()
          count += 1
        else:
          validation_img = directory.decode() + '/' + filename.decode()
          break
    
    if "Image_Pairs_Equal" in directory.decode() != -1:
        arr = np.ones((224,224, 3), dtype=int)
    else:
        arr = np.zeros((224,224, 3), dtype=int)
    
    return(preprocess(input_img), preprocess(validation_img), arr)

preprocessed_data = []
for element in data.as_numpy_iterator():
    
    '''
    print(element[0])
    for filename in os.listdir(element[0].decode()):
        print(element[0].decode() + '/' + filename)
    '''

    preprocessed_data.append(preprocess_twin(element[0], element[1]))

b = tf.convert_to_tensor(preprocessed_data)

In [ ]:
preprocessed_data

data_final = tf.data.Dataset.from_tensors(b)

In [ ]:
# Build dataloader pipeline
print(tf.executing_eagerly())
#data = data.map(preprocess_twin)
data_final = data_final.cache()
data_final = data_final.cache()
data_final = data_final.shuffle(buffer_size=10000)

# Training partition
train_data = data_final.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

# Testing partition
test_data = data_final.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

True


In [ ]:
siamese_model = make_siamese_model()
siamese_model.summary()
binary_cross_loss = tf.losses.BinaryCrossentropy()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation_img (InputLayer)    [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         463633728   ['input_img[0][0]',              
                                                                  'validation_img[0][

In [ ]:
opt = tf.keras.optimizers.Adam(1e-4) # 0.0001

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [ ]:
test_batch = train_data.as_numpy_iterator()

In [ ]:
batch_1 = test_batch.next()

In [ ]:
#X = batch_1[:2]

#(1, 300, 3, 224, 224, 3) is our data pipeline batch shape after pre processing
print(batch_1[0][0][0])
print(batch_1[0][0][1])
print(batch_1[0][0][2])

#This means we would want to iterate through our data as follows batch_1[0][iter][n] where iter is the dimension we iterate

[[[0.46866748 0.4916767  0.43969586]
  [0.35944375 0.4055022  0.3505002 ]
  [0.36060423 0.45514205 0.32765105]
  ...
  [0.00754305 0.15016012 0.03931506]
  [0.00896353 0.16334508 0.0242096 ]
  [0.00482188 0.15028018 0.03833527]]

 [[0.36012402 0.43561423 0.32743093]
  [0.35516208 0.41770712 0.35696283]
  [0.3690276  0.5188876  0.3679672 ]
  ...
  [0.00576215 0.14513765 0.05054046]
  [0.00392157 0.15258092 0.03503397]
  [0.00392157 0.11730656 0.04461879]]

 [[0.3487395  0.41194475 0.31522608]
  [0.34281713 0.4086835  0.30560225]
  [0.3639856  0.4707083  0.34371752]
  ...
  [0.05520188 0.18359266 0.09359695]
  [0.00386153 0.12184884 0.04915943]
  [0.00338138 0.1081227  0.02156876]]

 ...

 [[0.5331173  0.5295966  0.5475432 ]
  [0.6413787  0.62907445 0.6858768 ]
  [0.6101643  0.586595   0.61606616]
  ...
  [0.21698573 0.10938196 0.09395559]
  [0.15936351 0.07018778 0.05558259]
  [0.20730372 0.14359798 0.12859315]]

 [[0.6222102  0.55578464 0.59774095]
  [0.74547726 0.75488096 0.7811109 ]


In [ ]:
tf.losses.BinaryCrossentropy??

In [ ]:
@tf.function
def train_step(pair):
    
    # Record all of our operations 
    with tf.GradientTape() as tape:     
        # (1, 300, 3, 224, 224, 3) is our data pipeline batch shape after pre processing
        # Get anchor and positive/negative image
        X1 = pair[0]
        X2 = pair[1]
        # Get label
        y = pair[2]
        
        # Forward pass
        yhat = siamese_model(X1, X2, training=True)
        # Calculate loss
        loss = binary_cross_loss(y, yhat)
    print(loss)
        
    # Calculate gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    
    # Calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
        
    # Return loss
    return loss

In [ ]:
# Import metric calculations
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
def train(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Creating a metric object 
        r = Recall()
        p = Precision()
        
        # Loop through each batch
        for idx, batch in enumerate(data):
            for e in batch:
                for i in e:                    
                    # Run train step here
                    loss = train_step(i)
                    print(i[0])
                    print(i[1])
                    yhat = siamese_model.predict(i[0],i[1])
                    r.update_state(i[2], yhat)
                    p.update_state(i[2], yhat) 
                    progbar.update(idx+1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())
        
        # Save checkpoints
        if epoch % 10 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
EPOCHS = 50

In [ ]:
train(train_data, EPOCHS)


 Epoch 1/50


ValueError: ignored

In [ ]:
# Get a batch of test data
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

y_hat = siamese_model.predict([test_input, test_val])

print(y_hat)

# Post processing the results 
[1 if prediction > 0.5 else 0 for prediction in y_hat ]

# Creating a metric object 
m = Recall()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

# Creating a metric object 
m = Precision()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

r = Recall()
p = Precision()

for test_input, test_val, y_true in test_data.as_numpy_iterator():
    yhat = siamese_model.predict([test_input, test_val])
    r.update_state(y_true, yhat)
    p.update_state(y_true,yhat) 

print(r.result().numpy(), p.result().numpy())

# Set plot size 
plt.figure(figsize=(10,8))

# Set first subplot
plt.subplot(1,2,1)
plt.imshow(test_input[0])

# Set second subplot
plt.subplot(1,2,2)
plt.imshow(test_val[0])

# Renders cleanly
plt.show()

In [ ]:
# Save weights
siamese_model.save('/content/drive/MyDrive/Third_Year_Project/Siamese_Network/siamesemodelv2.h5')
L1Dist
# Reload model 
siamese_model = tf.keras.models.load_model('/content/drive/MyDrive/Third_Year_Project/Siamese_Network/siamesemodelv2.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

# Make predictions with reloaded model
siamese_model.predict([test_input, test_val])

In [ ]:
# View model summary
siamese_model.summary()

In [ ]:
import h5py

filename = "/content/drive/MyDrive/Third_Year_Project/Siamese_Network/siamesemodelv2.h5"

h5 = h5py.File(filename,'r')

futures_data = h5  # VSTOXX futures data

print(h5)

h5.close()